In [14]:
#!pip install langchain-community langchain-openai langchain-chroma langchain langchain-huggingface 

# !pip install langchain_google_genai
# !pip install langchain_groq 

# !pip install transformers


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement ChatGoogleGenerativeAI (from versions: none)
ERROR: No matching distribution found for ChatGoogleGenerativeAI


In [16]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
import os

In [17]:
path = "/work/AI-EHR/discharge_me/data/test_phase_2/all_text_data_by_hadm_id"
loader = DirectoryLoader(path, glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

FileNotFoundError: Directory not found: 'discharge_me/data/test_phase_2/all_text_data_by_hadm_id'

In [29]:
from config import GROQ_API_KEY, OPENAI_API_KEY2
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY2

In [4]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500, device = 0
)
hf = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.39s/it]


In [5]:
llm = HuggingFacePipeline.from_model_id(
    model_id="epfl-llm/meditron-7b",
    task="text-generation",
    device=0,
    pipeline_kwargs={"max_new_tokens": 500},
)

/home/mahajan.d/.conda/envs/ai-ehr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.40s/it]


In [6]:
context = """
 
History of Present Illness:
___ with ___ pemphigoid who presents with leg swelling and 
complaints of oral ulcers.  
 Patient is very difficult to get an accurate history from as he 
exhibits significant tangentiality and does not answer questions 
directly.  
 He states both legs are very swollen. Unclear how long this has 
been going on for. He was discharged from the ED in ___ for 
similar complaints. He was given a course of Keflex for 
cellulitis, but he states that the antibiotics never work.  
 He also complains of his bullous pemphigoid and how it has 
turned into lymphoma in his mouth. He also endorses diffuse 
lymphadenopathy. This has been going on for years. He has not 
seen a dermatologist in ___ in a long time (unclear just how 
long) because according to him not a single dermatologist in 
___ treats bullous pemphigoid. He says he has talked to ___ 
___ about getting his BP treated but has had trouble ever 
getting in touch with a doctor directly.  
 Patient states that his abdomen is swollen but cannot give a 
time course. He  
 He was referred into the ED by his PCP for evaluation of his 
leg lesions and abdominal swelling. His weight is 205 today, 
which is up from 162 in ___.  
 In the ED, initial vitals were: 97.9 92 116/72 22 97% RA 
93.2kgs  
 - Labs notable for WBC of 12.5  
 - Imaging: LLE US did not show evidence of DVT; CXR showed mild 
cardiomegaly and mild vascular congestion.  
 - Bedside US did not show any evidence of ascites.  
 Patient was given: Duonebs and CTX 1g IV for LLE cellulitis  
 
Past Medical History:
PAST MEDICAL HISTORY:  
- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory 
evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis  
"""

In [7]:
template = """
            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: {context}
            question: {input}
            answer:
            """
prompt = PromptTemplate.from_template(template)

In [8]:
question = "Given the History of Present Illness, What is the relevant past medical history?"
gpu_chain = prompt | hf
response = gpu_chain.invoke({"input": question, "context": context})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [9]:
print(response)


            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: 
 
History of Present Illness:
___ with ___ pemphigoid who presents with leg swelling and 
complaints of oral ulcers.  
 Patient is very difficult to get an accurate history from as he 
exhibits significant tangentiality and does not answer questions 
directly.  
 He states both legs are very swollen. Unclear how long this has 
been going on for. He was discharged from the ED in ___ for 
similar complaints. He was given a course of Keflex for 
cellulitis, but he states that the antibiotics never work.  
 He also complains of his bullous pemphigoid and how it has 
turned into lymphoma in his mouth. He also endorses diffuse 
lymphadenopathy. This has been going on for

In [8]:
from config import GOOGLE_API_KEY2
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY2

In [36]:
chat = ChatGroq(
    temperature=0,
    model="llama3-8b-8192",
    # api_key="" # Optional if not set as an environment variable
)

In [37]:
def generate_response(retriever, query):

    try:
        # llm = ChatGoogleGenerativeAI(model="gemini-pro")
        # llm = ChatOpenAI(model="gpt-4")
        llm =chat
#         llm = HuggingFacePipeline.from_model_id(
#     model_id="epfl-llm/meditron-7b",
#     task="text-generation",
# )
        template = """
            You are a helpful Medical AI assistant. Answer based on the context provided. 
            context: {context}
            input: {input}
            answer:
            """
        prompt = PromptTemplate.from_template(template)
        combine_docs_chain = create_stuff_documents_chain(llm, prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        response = retrieval_chain.invoke({"input": query})
        # print(response["answer"])
        return response["answer"]
    except Exception as e:
        print(f"An error occurred while generating the response: {e}")
        return None

In [27]:
questions = [
"Given the History of Present Illness, What is the relevant past medical history?",
"Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?",
"Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?",
"Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?",
"given the history of present illness, What was the initial treatment course?",
"Given the History of Present illness, What was the initial diagnostic work up and pertinent results?",
"Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? Summarize the impressions from each study. Make a table of Imaging study, indication, and impression."
]

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """You are an internal medicine physician conducting a review of a patient's EHR. 
        You need to find relevant and accurate information contained in the text of clinical notes. 
        Using the information contained in the context provided, answer the question about the patient's medical records..
Respond only to the question asked, response should be concise and relevant to the question.
If the answer cannot be deduced from the context, respond that the information can not be deduced from the information provided.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

In [21]:
response = generate_response(retriever, questions[0])
print(response)

- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory 
evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis


In [38]:
for question in questions:
    response = generate_response(retriever, question)
    print(question,"\n")
    print(response)
    print("\n*********************")

An error occurred while generating the response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Given the History of Present Illness, What is the relevant past medical history? 

None

*********************
An error occurred while generating the response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the pat